In [9]:
import plotly
import plotly.express as px
import numpy as np

from class_definitions import *
import pandas as pd
from vivino_db import database
db = database.db
db.db_engine.table_names()

In [2]:
tables = {}
for table in db.db_engine.table_names():
    tables[table]=pd.read_sql_table(table,db.db_engine)

In [3]:
df = pd.read_csv('dataset.csv')

### Some updates and formatting that I forgot when compiling the data

In [4]:
region_country = pd.read_sql('select name as region,country_name from regions',db.db_engine)
df = pd.merge(df,region_country,on='region')

stmt="""
SELECT
    v.id,
    w.type_id
FROM
    vintages v
INNER JOIN
    wines w on v.wine_id = w.id
"""
types = pd.read_sql(stmt,db.db_engine)

df = pd.merge(df,types,on='id')

In [5]:
type_map = {
    1:'Red',
    2:'White',
    3:'Sparkling',
    4:'Rose'
}
df['type']=df['type_id'].map(type_map)

In [7]:
# messy manual updates for now
# need a better way to match vivino results with wineSearcher results
df.loc[31,'type_id']=3
df.loc[158,['Producer','Region/Appellation','Grape/Blend','Wine Style','Food Suggestion']] = ['Chateu Gassier', 'Cotes de Provence', 'Rare Rose Blend', 'Rose - Crisp and Dry', 'Tomato-based Dishes']
df.loc[53,['Producer','Region/Appellation','Grape/Blend','Wine Style','Food Suggestion']] = ['Weingut Karl Proidl',"Kremstal",'Riesling',"White - Green and Flinty", 'White Fish' ]
df.loc[4,['Producer','Region/Appellation','Grape/Blend','Wine Style','Food Suggestion']] = ['Weingut Koehler-Ruprecht','Pfalz','Pinot Noir','Red - Light and Perfumed','Meaty and Oily Fish']

In [29]:
i = []
for t in type_map.values():
    i.extend(df[(df['Wine Style'].isna()==False)&(df['Wine Style'].str.contains(t))&(df['type']!=t)].index)

temp = df.copy(deep=True) 
temp = df.drop(index=i)[df['Wine Style'].isna()==False]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [30]:
temp['Wine Style']=temp.apply(lambda x: x['Wine Style'].split('-')[-1].strip(),axis=1)

In [40]:
fig = px.sunburst(temp,path=['type','Wine Style','Grape/Blend'],
                  color='acidity',color_continuous_scale='PuBu',
                  color_continuous_midpoint=np.average(temp['acidity'])
                 )
fig.show(renderer='browser')

In [78]:
tasting_notes=joblib.load('Wines/tasting_notes.pkl')

In [3]:
t = tasting_notes[0]

In [4]:
t

{4470482: [{'index': 0,
   'tag': 'blackcurrant',
   'note': '2 mentions of black fruit notes'},
  {'index': 0, 'tag': 'plum', 'note': '2 mentions of black fruit notes'},
  {'index': 1, 'tag': 'prune', 'note': '1 mentions of dried fruit notes'},
  {'index': 2, 'tag': 'violet', 'note': '1 mentions of floral notes'},
  {'index': 3, 'tag': 'chard', 'note': '1 mentions of vegetal notes'},
  {'index': 3, 'tag': 'red beet', 'note': '1 mentions of vegetal notes'},
  {'index': 4, 'tag': 'chocolate', 'note': '1 mentions of oaky notes'}]}

In [5]:
records = []
for t in tasting_notes:
    d = {}
    for k,v in t.items():
        d['id']=k
        d['notes']=list({x['note'] for x in v})
    records.append(d)

In [7]:
unique_notes = set()
for record in records:
    notes = [x.split(' mentions of ')[-1].replace(' notes','') for x in record['notes']]
    [unique_notes.add(x) for x in notes]

In [8]:
unique_notes

{'ageing',
 'black fruit',
 'citrus',
 'dried fruit',
 'earthy',
 'floral',
 'microbio',
 'oaky',
 'red fruit',
 'spices',
 'tree fruit',
 'tropical',
 'vegetal'}

In [23]:
df['style'].value_counts()

Northern Italy Red             9
German Riesling                7
Italian Nebbiolo               6
Northern Italy White           5
New Zealand Sauvignon Blanc    4
                              ..
Burgundy Côte de Nuits Red     1
Burgundy Macônnais White       1
Oregon Chardonnay              1
Italian Brunello               1
Australian Riesling            1
Name: style, Length: 85, dtype: int64

In [30]:
df['varietal_name'].value_counts()

Red                        26
White                      20
Riesling                   12
Chardonnay                 10
Nebbiolo                    6
Sauvignon Blanc             5
Cabernet Sauvignon          5
Champagne                   4
Chianti                     3
Chablis                     3
Syrah                       3
Pinot Noir                  3
Barolo                      3
Red Blend                   3
Cava                        3
Pinot Blanc                 2
Prosecco                    2
Côte de Beaune Red          2
Grüner Veltliner            2
Pinot Grigio                2
Albariño                    2
Spätburgunder               2
Montepulciano d'Abruzzo     2
Priorat Red                 2
Bordeaux Blend              2
Cremant                     1
Silvaner                    1
Merlot                      1
Muscadet                    1
Macônnais White             1
Verdejo                     1
Grenache                    1
Dão Red                     1
Bonarda   

In [41]:
fig = px.sunburst(df,path=['country_name','style'])

In [42]:
fig.show(renderer='browser')

In [21]:
df['style']

0                   Austrian Zweigelt
1                    Italian Nebbiolo
2      Californian Cabernet Sauvignon
3                German Spätburgunder
4                German Spätburgunder
                    ...              
154        Languedoc-Roussillon White
155                     Italian Soave
156                Northern Italy Red
157                     Provence Rosé
158             German Müller Thurgau
Name: style, Length: 159, dtype: object

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160 entries, 0 to 159
Data columns (total 29 columns):
name                   160 non-null object
Producer               125 non-null object
Region/Appellation     139 non-null object
Grape/Blend            139 non-null object
Food Suggestion        135 non-null object
Wine Style             135 non-null object
Alcohol Content        85 non-null object
instance               160 non-null object
vivino_wine            160 non-null object
id                     160 non-null int64
winery                 160 non-null object
region                 160 non-null object
style                  160 non-null object
varietal_name          160 non-null object
regional_name          160 non-null object
body                   160 non-null int64
acidity                160 non-null int64
style_description      119 non-null object
ratings_count          160 non-null int64
ratings_average        160 non-null float64
vintage_description    0 non-null float